In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#TF stuff
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
import keras_tuner as kt # for tuner

from sklearn.metrics import log_loss

In [ ]:
#https://drive.google.com/drive/folders/1Kogxpks1YoveJfzjmsztlLUyplIP00B8?usp=sharing
train_features = pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
train_targets = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')

train_features_enc = pd.get_dummies(train_features, columns=['cp_type', 'cp_dose'], drop_first=True)

X = train_features_enc.iloc[:,1:].to_numpy()
y = train_targets.iloc[:,1:].to_numpy() 

In [ ]:
def l3_res_model(input_shape, no_classes, lr):
    inputs = tf.keras.Input(shape=input_shape)
    x = layers.Dense(120, activation='sigmoid')(inputs)
    x = layers.BatchNormalization()(x)
    b_1 = layers.Dropout(0.25)(x)
    x = layers.Dense(120, activation='sigmoid')(b_1)
    x = layers.BatchNormalization()(x)
    b_2 = layers.Dropout(0.25)(x)
    x = layers.Dense(120, activation='sigmoid')(b_2)
    x = layers.BatchNormalization()(x)
    b_3 = layers.Dropout(0.25)(x)
    tot_op = tf.keras.layers.add([b_1, b_2, b_3])
    outputs = layers.Dense(no_classes, activation='sigmoid')(tot_op)
    model = tf.keras.Model(inputs, outputs)
    model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate = lr), metrics=['binary_crossentropy'])
    return model

In [ ]:
tf.random.set_seed(1010)
np.random.seed(1010)

#nnclf = l3_res_model()

X_train, y_train = X, y

control_vehicle_mask = X_train[:,-2] == 0
X_train = X_train[~control_vehicle_mask,:]
y_train = y_train[~control_vehicle_mask]

nnclf = l3_res_model((875,),206,0.0006)
nnclf2 = l3_res_model((875,),206,0.0001)
nnclf3 = l3_res_model((875,),206,0.001)
nnclf4 = l3_res_model((875,),206,0.0004)
nnclf5 = l3_res_model((875,),206,0.0003)
hist = nnclf.fit(X_train, y_train, batch_size=512, epochs=55, verbose=0)
hist2 = nnclf2.fit(X_train, y_train, batch_size=512, epochs=55, verbose=0)
hist3 = nnclf3.fit(X_train, y_train, batch_size=512, epochs=55, verbose=0)
hist4 = nnclf4.fit(X_train, y_train, batch_size=512, epochs=55, verbose=0)
hist5 = nnclf5.fit(X_train, y_train, batch_size=512, epochs=55, verbose=0)

preds = nnclf.predict(X)

control_mask = X[:,-2]==0
preds[control_mask] = 0
loss = log_loss(np.ravel(y), np.ravel(preds))
print('In sample Loss: '+str(loss))

In [ ]:
test_features = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')
test_features_enc = pd.get_dummies(test_features, columns=['cp_type', 'cp_dose'], drop_first=True)

X_test = test_features_enc.iloc[:,1:].to_numpy()

test_preds1 = nnclf.predict(X_test)
test_preds2 = nnclf2.predict(X_test)
test_preds3 = nnclf3.predict(X_test)
test_preds4 = nnclf4.predict(X_test)
test_preds5 = nnclf5.predict(X_test)
test_preds = (test_preds1+test_preds2+test_preds3+test_preds4+test_preds5)/5

control_mask_test = X_test[:,-2]==0
test_preds[control_mask_test] = 0

final_submission = test_features[['sig_id']] #get id column

columns_names = list(train_targets.columns)
temp = pd.DataFrame(test_preds, columns = columns_names[1:]) #dataframe of prediction output

final_submission = pd.concat([final_submission,temp], axis=1) #adding id column
final_submission

In [ ]:
final_submission.to_csv('submission.csv', index=False)